# Decision Tree Calssifier:

In [17]:
#imports
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

In [ ]:
#reading csv files from download
df = pd.read_csv('/../../isic-2024-challenge/train-metadata.csv', low_memory=False)


missing_percentages = (df.isnull().sum() / len(df)) * 100
columns_to_keep = missing_percentages[missing_percentages < 50].index
skin_cancer_df= df[columns_to_keep]
skin_cancer_df = skin_cancer_df.fillna(skin_cancer_df.mode().iloc[0])
skin_cancer_df.shape

skin_cancer_df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'isic-2024-challenge/data/train-metadata.csv'

In [ ]:
# Distribution of the target variable
skin_cancer_df['target'].value_counts()

# not balanced since there are more counts of 0 than 1 -> maybe struggle to learn from minorityÍ

target
0    400666
1       393
Name: count, dtype: int64

In [ ]:
# Separate features (X) and target (y)
X = skin_cancer_df.drop(columns=['target'])  # Replace 'target' with your actual column name
y = skin_cancer_df['target']

In [ ]:
# Check for non-numeric columns
non_numeric_columns = X.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)


Non-numeric columns: Index(['isic_id', 'patient_id', 'sex', 'anatom_site_general', 'image_type',
       'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple',
       'attribution', 'copyright_license', 'iddx_full', 'iddx_1'],
      dtype='object')


In [ ]:
# (encode the non-numeric data into numeric values using)


# Initialize a LabelEncoder to assign an integer to each category.
label_encoders = {}

# Encode each non-numeric column
for column in non_numeric_columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le 

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35, stratify=y)

In [ ]:
# Apply SMOTE to balance the training set
smote = SMOTE(random_state=35)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Class distribution after SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Class distribution after SMOTE:
target
0    320533
1    320533
Name: count, dtype: int64


In [ ]:
# "skin_cancer_decisiontree" is instantiated as an "object" of DecisionTreeClassifier "class". 
# Train a DecisionTreeClassifier on the resampled dataset
skin_cancer_decisiontree = DecisionTreeClassifier(class_weight=None, max_depth=10, min_samples_split=5, random_state=35)


In [ ]:
#use the method "fit" of the objects "my_decisiontree" along with training dataset and labels to train the model.

skin_cancer_decisiontree.fit(X_train_resampled, y_train_resampled)

DecisionTreeClassifier(max_depth=10, min_samples_split=5, random_state=35)

In [ ]:
# Testing on the testing set:

y_predict = skin_cancer_decisiontree.predict(X_test)

print(y_predict)

[0 0 0 ... 0 0 0]


In [ ]:
# compare the "predicted labels" for the Testing Set with its "actual labels"
accuracy = accuracy_score(y_test, y_predict)

print(accuracy)


1.0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     80133
           1       1.00      1.00      1.00        79

    accuracy                           1.00     80212
   macro avg       1.00      1.00      1.00     80212
weighted avg       1.00      1.00      1.00     80212



In [ ]:
from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
scores = cross_val_score(skin_cancer_decisiontree, X, y, cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Mean cross-validation accuracy:", scores.mean())


Cross-validation scores: [1. 1. 1. 1. 1.]
Mean cross-validation accuracy: 1.0
